In [ ]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import time
from sklearn import metrics
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
import seaborn as sns
import xgboost as xgb
import sklearn.linear_model as lm
from sklearn import svm
from xgboost import plot_importance
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import joblib
import tensorflow as tf

In [ ]:
import os
 
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:


ndatas_df=pd.read_csv(r'D:\\nonlandslide.csv')

pdatas_df=pd.read_csv(r'D:\\landslide.csv',encoding='ISO-8859-1')


In [ ]:
# ndatacqx['label']=0
# pdatacqx['label']=1

ndatas_df['label']=0

pdatas_df['label']=1


In [ ]:
def shuf(i,n):
    i=shuffle(i[['Lithology',
       'Distance_From_Faults', 'distance_from_water', 'Distance_From_Road',
       'SWI', 'RDLS', 'Slope', 'Aspect', 'Profile_Curvature',
       'Plane_Curvature', 'Elevation', 'ndviwithcq', 'Soil_Type', 'Landuse',
       'Population', 'Aridity', 'GDP', 'IM', 'Annually_Mean_Precipitation',
       'Annually_Mean_Temperature','label']],random_state=67).dropna().reset_index().drop(columns=['index'])[:len(n)]
    return i

In [ ]:
ndatas_df_new=shuf(ndatas_df,pdatas_df)
pdatas_df_new=shuf(pdatas_df,ndatas_df_new)

# Source domain

In [ ]:
data_df=ndatas_df_new.append(pdatas_df_new)
data_df=shuffle(data_df).reset_index().drop(columns=['index'])

In [ ]:
import numpy as np
from tensorflow import keras
from keras import backend as K
def cosine_decay_with_warmup(global_step,
                             learning_rate_base,
                             total_steps,
                             warmup_learning_rate=0.0,
                             warmup_steps=0,
                             hold_base_rate_steps=0):

    if total_steps < warmup_steps:
        raise ValueError('total_steps must be larger or equal to '
                         'warmup_steps.')
  
    learning_rate = 0.5 * learning_rate_base * (1 + np.cos(np.pi *
        (global_step - warmup_steps - hold_base_rate_steps) / float(total_steps - warmup_steps - hold_base_rate_steps)))

    if hold_base_rate_steps > 0:
        learning_rate = np.where(global_step > warmup_steps + hold_base_rate_steps,
                                 learning_rate, learning_rate_base)
    if warmup_steps > 0:
        if learning_rate_base < warmup_learning_rate:
            raise ValueError('learning_rate_base must be larger or equal to '
                             'warmup_learning_rate.')

        slope = (learning_rate_base - warmup_learning_rate) / warmup_steps
        warmup_rate = slope * global_step + warmup_learning_rate

        learning_rate = np.where(global_step < warmup_steps, warmup_rate,
                                 learning_rate)
    return np.where(global_step > total_steps, 0.0, learning_rate)
class WarmUpCosineDecayScheduler(keras.callbacks.Callback):
  
    def __init__(self,
                 learning_rate_base,
                 total_steps,
                 global_step_init=0,
                 warmup_learning_rate=0.0,
                 warmup_steps=0,
                 hold_base_rate_steps=0,
                 verbose=0):
        super(WarmUpCosineDecayScheduler, self).__init__()
        self.learning_rate_base = learning_rate_base
        self.total_steps = total_steps
        self.global_step = global_step_init
        self.warmup_learning_rate = warmup_learning_rate
        self.warmup_steps = warmup_steps
        self.hold_base_rate_steps = hold_base_rate_steps
        self.verbose = verbose
 
        self.learning_rates = []

    def on_batch_end(self, batch, logs=None):
        self.global_step = self.global_step + 1
        lr = K.get_value(self.model.optimizer.lr)
        self.learning_rates.append(lr)

    def on_batch_begin(self, batch, logs=None):
        lr = cosine_decay_with_warmup(global_step=self.global_step,
                                      learning_rate_base=self.learning_rate_base,
                                      total_steps=self.total_steps,                                      warmup_learning_rate=self.warmup_learning_rate,
                                      warmup_steps=self.warmup_steps,                                    hold_base_rate_steps=self.hold_base_rate_steps)
        K.set_value(self.model.optimizer.lr, lr)
        if self.verbose > 0:
            print('\nBatch %05d: setting learning '
                  'rate to %s.' % (self.global_step + 1, lr))

In [ ]:
data=data_df.iloc[:,:-1]
label=data_df.iloc[:,-1]
feat_labels=data_df.columns[:-1]

In [ ]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(data,label,test_size=0.2)

In [ ]:
mean=Xtrain.mean(axis=0)
std=Xtrain.std(axis=0)

In [ ]:
Xtrain=(Xtrain-mean)/std
Xtest=(Xtest-mean)/std

In [ ]:
# Xtraincq=(Xtraincq-cqmean)/cqstd
# Xtestcq=(Xtestcq-cqmean)/cqstd

In [ ]:
Xtrain=np.expand_dims(Xtrain,-1)
Xtest=np.expand_dims(Xtest,-1)

In [ ]:
keras=tf.keras
layers=tf.keras.layers

In [ ]:
model=keras.Sequential()

model.add(layers.Conv1D(64,3,activation='relu',padding='same'))
model.add(layers.MaxPooling1D(2))
model.add(layers.Dropout(0.1))

model.add(layers.Conv1D(128,3,activation='relu',padding='same'))
model.add(layers.MaxPooling1D(2))
model.add(layers.Dropout(0.1))

model.add(layers.Conv1D(256,3,activation='relu',padding='same'))
model.add(layers.MaxPooling1D(2))
model.add(layers.Dropout(0.1))

model.add(layers.Conv1D(512,3,activation='relu',padding='same'))
model.add(layers.MaxPooling1D(2))
model.add(layers.Dropout(0.1))

model.add(layers.Bidirectional(layers.LSTM(256,return_sequences=True)))
model.add(layers.Dropout(0.2))

model.add(layers.Bidirectional(layers.LSTM(128,return_sequences=True)))
model.add(layers.Dropout(0.2))



model.add(layers.Dense(128))
model.add(layers.Dense(64))
model.add(layers.Dense(1,activation='sigmoid'))

In [ ]:
# #8087

# model=keras.Sequential()
# model.add(layers.Conv1D(32,1,input_shape=(Xtrain.shape[1:]),activation='relu',padding='same'))
# model.add(layers.Conv1D(32,3,input_shape=(Xtrain.shape[1:]),activation='relu',padding='same'))
# model.add(layers.Conv1D(32,5,input_shape=(Xtrain.shape[1:]),activation='relu',padding='same'))
# model.add(layers.BatchNormalization())
# model.add(layers.MaxPooling1D(2))
# model.add(layers.Dropout(0.3))

# model.add(layers.Conv1D(64,5,activation='relu',padding='same'))
# model.add(layers.Conv1D(64,5,activation='relu',padding='same'))
# model.add(layers.BatchNormalization())
# model.add(layers.MaxPooling1D(2))
# model.add(layers.Dropout(0.5))

# model.add(layers.Conv1D(128,5,activation='relu',padding='same'))
# model.add(layers.Conv1D(128,5,activation='relu',padding='same'))
# model.add(layers.BatchNormalization())
# model.add(layers.MaxPooling1D(2))
# model.add(layers.Dropout(0.6))


# model.add(layers.Conv1D(256,3,activation='relu',padding='same'))
# model.add(layers.Conv1D(256,3,activation='relu',padding='same'))
# model.add(layers.BatchNormalization())
# model.add(layers.MaxPooling1D(2))
# model.add(layers.Dropout(0.6))

# model.add(layers.Conv1D(512,1,activation='relu',padding='same'))
# model.add(layers.BatchNormalization())
# model.add(layers.Dropout(0.5))

# model.add(layers.Bidirectional(layers.LSTM(256,return_sequences=True)))
# model.add(layers.Dropout(0.5))

# model.add(layers.Bidirectional(layers.LSTM(128,return_sequences=True)))
# model.add(layers.Dropout(0.5))

# model.add(layers.Bidirectional(layers.LSTM(128)))
# model.add(layers.Dropout(0.5))


# model.add(layers.Dense(64))
# model.add(layers.Dense(32))
# model.add(layers.Dense(16))
# model.add(layers.Dense(1,activation='sigmoid'))

In [ ]:
model.compile(optimizer=keras.optimizers.RMSprop(),
             loss='binary_crossentropy',
             metrics='AUC',
             
             )

In [ ]:
Xtrain.shape

In [ ]:

sample_count = Xtrain.shape[0]
# Total epochs to train.
epochs = 10
# Number of warmup epochs.
warmup_epoch = 5
# Training batch size, set small value here for demonstration purpose.
batch_size = 512
# Base learning rate after warmup.
learning_rate_base = 0.001

total_steps = int(epochs * sample_count / batch_size)
# Compute the number of warmup batches.
warmup_steps = int(warmup_epoch * sample_count / batch_size)



# Compute the number of warmup batches.
warmup_batches = warmup_epoch * sample_count / batch_size

# Create the Learning rate scheduler.
warm_up_lr = WarmUpCosineDecayScheduler(learning_rate_base=learning_rate_base,
                                            total_steps=total_steps,
                                            warmup_learning_rate=4e-06,
                                            warmup_steps=warmup_steps,
                                            hold_base_rate_steps=5,
                                            )

# Train the model, iterating on the data in batches of 32 samples


# import matplotlib.pyplot as plt
# plt.plot(warm_up_lr.learning_rates)
# plt.xlabel('Step', fontsize=20)
# plt.ylabel('lr', fontsize=20)
# plt.axis([0, total_steps, 0, learning_rate_base*1.1])
# plt.xticks(np.arange(0, epochs, 1))
# plt.grid()
# plt.title('Cosine decay with warmup', fontsize=20)
# plt.show()


In [ ]:
history=model.fit(Xtrain, Ytrain, epochs=100, batch_size=512, callbacks=[warm_up_lr],validation_data=(Xtest,Ytest))

In [ ]:
import matplotlib.pyplot as plt
plt.plot(warm_up_lr.learning_rates)
plt.xlabel('Step', fontsize=20)
plt.ylabel('lr', fontsize=20)
plt.axis([0, total_steps, 0, learning_rate_base*1.1])
plt.xticks(np.arange(0, epochs, 1))
plt.grid()
plt.title('Cosine decay with warmup', fontsize=20)
plt.show()

In [ ]:
# model.save(r'D:Source_Model.h5') 

# Target


In [ ]:
ndatacqd=pd.read_csv(r'D:\\Target\\Nonlandslide.csv')
pdatacqd=pd.read_csv(r'D:\\Target\\Landslide.csv',encoding='ISO-8859-1')

In [ ]:
ndatacqd['label']=0
pdatacqd['label']=1

In [ ]:
ndatacqd=ndatacqd.rename(columns={'Distance_from_Faultscq':'Distance_From_Faults'})
pdatacqd=pdatacqd.rename(columns={'Distance_from_Faultscq':'Distance_From_Faults'})

In [ ]:

ndata_cqd_new=shuf(ndatacqd,pdatacqd)
pdata_cqd_new=shuf(pdatacqd,ndata_cqd_new)

In [ ]:

data_cqd=ndata_cqd_new.append(pdata_cqd_new)
data_cqd=shuffle(data_cqd).reset_index().drop(columns=['index'])

In [ ]:
cqddata=data_cqd.iloc[:,:-1]
cqdlabel=data_cqd.iloc[:,-1]
cqdfeat_labels=data_cqd.columns[:-1]

In [ ]:
Xtraincqd, Xtestcqd, Ytraincqd, Ytestcqd = train_test_split(cqddata,cqdlabel,test_size=0.2)

In [ ]:
cqdmean=Xtraincqd.mean(axis=0)
cqdstd=Xtraincqd.std(axis=0)

In [ ]:
# Xtraincqd=(Xtraincqd-mean[:len(Xtraincqd)])/std[:len(Xtraincqd)]
# Xtestcqd=(Xtestcqd-mean[:len(Xtraincqd)])/std[:len(Xtraincqd)]

In [ ]:
Xtraincqd=(Xtraincqd-cqdmean)/cqdstd
Xtestcqd=(Xtestcqd-cqdmean)/cqdstd

In [ ]:
Xtraincqd=np.expand_dims(Xtraincqd,-1)
Xtestcqd=np.expand_dims(Xtestcqd,-1)

In [ ]:
conv=tf.keras.models.load_model(r'D:Source_Model.h5')

In [ ]:
for i in range(5):
    conv.layers[i].trainable=False


In [ ]:
model=keras.Sequential(conv)

In [ ]:
model.compile(optimizer=keras.optimizers.RMSprop(),
             loss='binary_crossentropy',
             metrics='AUC',        
             )

In [ ]:

sample_count = Xtraincqd.shape[0]
# Total epochs to train.
epochs = 100
# Number of warmup epochs.
warmup_epoch = 5
# Training batch size, set small value here for demonstration purpose.
batch_size = 512
# Base learning rate after warmup.
learning_rate_base = 0.001

total_steps = int(epochs * sample_count / batch_size)
# Compute the number of warmup batches.
warmup_steps = int(warmup_epoch * sample_count / batch_size)



# Compute the number of warmup batches.
warmup_batches = warmup_epoch * sample_count / batch_size

# Create the Learning rate scheduler.
warm_up_lr_target = WarmUpCosineDecayScheduler(learning_rate_base=learning_rate_base,
                                            total_steps=total_steps,
                                            warmup_learning_rate=4e-06,
                                            warmup_steps=warmup_steps,
                                            hold_base_rate_steps=5,
                                            )

# Train the model, iterating on the data in batches of 32 samples


# import matplotlib.pyplot as plt
# plt.plot(warm_up_lr.learning_rates)
# plt.xlabel('Step', fontsize=20)
# plt.ylabel('lr', fontsize=20)
# plt.axis([0, total_steps, 0, learning_rate_base*1.1])
# plt.xticks(np.arange(0, epochs, 1))
# plt.grid()
# plt.title('Cosine decay with warmup', fontsize=20)
# plt.show()


In [ ]:
history_target=model.fit(Xtraincqd, Ytraincqd, epochs=100, batch_size=512,callbacks=[warm_up_lr_target],validation_data=(Xtestcqd,Ytestcqd))

In [ ]:
cq_=pd.read_csv(r'D:\\cqfishnet.csv')

In [ ]:
cq_=cq_.dropna()

In [ ]:
cq_data=cq_[['Lithology',
       'Distance_From_Faults', 'distance_from_water', 'Distance_From_Road',
       'SWI', 'RDLS', 'Slope', 'Aspect', 'Profile_Curvature',
       'Plane_Curvature', 'Elevation', 'ndviwithcq', 'Soil_Type', 'Landuse',
       'Population', 'Aridity', 'GDP', 'IM', 'Annually_Mean_Precipitation',
       'Annually_Mean_Temperature']]

In [ ]:
cq_mean=cq_data.mean(axis=0)
cq_std=cq_data.std(axis=0)

In [ ]:
cq_data=(cq_data-cq_mean)/cq_std


In [ ]:
cq_data=np.expand_dims(cq_data,-1)

In [ ]:
susceptibility5_=model.predict(cq_data)

In [ ]:
susceptibility5_df_=pd.DataFrame(susceptibility5_)

In [ ]:
cq_['susceptibility5_dl']=susceptibility5

In [ ]:
susceptibility5_df_dl=cq_[['OBJECTID','susceptibility5']]

In [ ]:
susceptibility5_df_dl.to_csv(r'D:\\susceptibility5.csv')